In [3]:
import pandas as pd
df = pd.read_csv('Nutrition__Physical_Activity__and_Obesity.csv')
print(df.head())

   YearStart  YearEnd LocationAbbr LocationDesc  \
0       2020     2020           US     National   
1       2014     2014           GU         Guam   
2       2013     2013           US     National   
3       2013     2013           US     National   
4       2015     2015           US     National   

                                   Datasource                    Class  \
0  Behavioral Risk Factor Surveillance System        Physical Activity   
1  Behavioral Risk Factor Surveillance System  Obesity / Weight Status   
2  Behavioral Risk Factor Surveillance System  Obesity / Weight Status   
3  Behavioral Risk Factor Surveillance System  Obesity / Weight Status   
4  Behavioral Risk Factor Surveillance System        Physical Activity   

                          Topic  \
0  Physical Activity - Behavior   
1       Obesity / Weight Status   
2       Obesity / Weight Status   
3       Obesity / Weight Status   
4  Physical Activity - Behavior   

                                     

In [5]:
df.describe()

,YearStart,YearEnd,Data_Value_Unit,Data_Value,Data_Value_Alt,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,LocationID
count,88629.000000,88629.000000,0.0,79851.000000,79851.000000,79851.000000,79851.000000,79851.000000,88629.000000
mean,2016.011362,2016.011362,NaN,31.236493,31.236493,26.907732,36.135953,3656.809044,30.906475
std,3.121080,3.121080,NaN,10.113829,10.113829,9.907896,11.061591,18706.392637,17.485456
min,2011.000000,2011.000000,NaN,0.900000,0.900000,0.300000,3.000000,50.000000,1.000000
25%,2013.000000,2013.000000,NaN,24.300000,24.300000,20.000000,28.600000,516.000000,17.000000
50%,2016.000000,2016.000000,NaN,31.200000,31.200000,26.800000,35.900000,1109.000000,30.000000
75%,2019.000000,2019.000000,NaN,37.000000,37.000000,32.900000,42.200000,2408.000000,45.000000
max,2021.000000,2021.000000,NaN,77.600000,77.600000,70.200000,87.700000,476876.000000,78.000000


In [7]:
df.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'Class', 'Topic', 'Question', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Alt', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Low_Confidence_Limit', 'High_Confidence_Limit ',
       'Sample_Size', 'Total', 'Age(years)', 'Education', 'Gender', 'Income',
       'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'LocationID', 'StratificationCategory1',
       'Stratification1', 'StratificationCategoryId1', 'StratificationID1'],
      dtype='object')

In [12]:
import pandas as pd
import numpy as np

# 1. Load the dataset (Make sure filename matches your upload)
df = pd.read_csv('Nutrition__Physical_Activity__and_Obesity.csv', low_memory=False)
df.columns = [c.strip() for c in df.columns]

# 2. Define keywords (Yours were perfect!)
indicator_keywords = {
    'low_fruit': ['fruit'],
    'low_veg': ['vegetable', 'veg'],
    'no_exercise': ['no leisure-time physical activity', 'physically inactive'],
    'obesity': ['obesity', 'obese', 'percent of adults with obesity']
}

def label_indicator(question):
    if not isinstance(question, str): return None
    q = question.lower()
    for ind, kws in indicator_keywords.items():
        for kw in kws:
            if kw in q: return ind
    return None

# 3. Apply labels and clean numerical data
df['indicator'] = df['Question'].apply(label_indicator)
df['Data_Value'] = pd.to_numeric(df['Data_Value'], errors='coerce')

# Filter out rows that aren't relevant
labeled = df.dropna(subset=['indicator', 'Data_Value']).copy()
labeled['Year'] = labeled['YearStart']

# 4. THE FIX: Group by DEMOGRAPHICS too, not just State
# We include 'StratificationCategory1' (e.g., Age) and 'Stratification1' (e.g., 18-24)
grouped = labeled.groupby([
    'LocationDesc',
    'Year',
    'StratificationCategory1',
    'Stratification1',
    'indicator'
])['Data_Value'].mean().reset_index()

# 5. Pivot so Metrics become columns (Easier to use in React)
pivot = grouped.pivot_table(
    index=['LocationDesc', 'Year', 'StratificationCategory1', 'Stratification1'],
    columns='indicator',
    values='Data_Value'
).reset_index()

# 6. Filter for the latest data (2021) to keep the file small for your web app
latest_pivot = pivot[pivot['Year'] == 2021].copy()

# Rename columns to be JS-friendly
latest_pivot.columns.name = None # remove index name
latest_pivot = latest_pivot.rename(columns={
    'LocationDesc': 'state',
    'StratificationCategory1': 'category',
    'Stratification1': 'group'
})

# Fill NaN with 0 or null (optional, JSON handles nulls fine)
# Save
latest_pivot.to_csv('clean_obesity_demographics.csv', index=False)
print('Saved! Rows:', len(latest_pivot))
print(latest_pivot.head())

Saved! Rows: 1388
       state  Year     category    group  low_fruit  low_veg  no_exercise  \
251  Alabama  2021  Age (years)  18 - 24       44.8     24.7         19.3   
252  Alabama  2021  Age (years)  25 - 34       49.4     20.3         23.3   
253  Alabama  2021  Age (years)  35 - 44       44.5     16.7         22.4   
254  Alabama  2021  Age (years)  45 - 54       49.2     19.5         35.6   
255  Alabama  2021  Age (years)  55 - 64       48.4     21.2         39.1   

     obesity  
251     25.4  
252     46.6  
253     45.8  
254     46.4  
255     44.4  


In [13]:
data = pd.read_csv('clean_obesity_demographics.csv')
data.head()


,state,Year,category,group,low_fruit,low_veg,no_exercise,obesity
0,Alabama,2021,Age (years),18 - 24,44.8,24.7,19.3,25.4
1,Alabama,2021,Age (years),25 - 34,49.4,20.3,23.3,46.6
2,Alabama,2021,Age (years),35 - 44,44.5,16.7,22.4,45.8
3,Alabama,2021,Age (years),45 - 54,49.2,19.5,35.6,46.4
4,Alabama,2021,Age (years),55 - 64,48.4,21.2,39.1,44.4


In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import json
from google.colab import files

print("🚀 Starting Data Processing...")

# 1. LOAD DATA
# ---------------------------------------------------------
try:
    df = pd.read_csv('Nutrition__Physical_Activity__and_Obesity.csv', low_memory=False)
except FileNotFoundError:
    print("❌ Error: Please upload 'Nutrition__Physical_Activity__and_Obesity.csv' first!")
    raise

df.columns = [c.strip() for c in df.columns]

# 2. CLEAN & LABEL
# ---------------------------------------------------------
indicator_keywords = {
    'low_fruit': ['fruit'],
    'no_exercise': ['no leisure-time physical activity', 'physically inactive'],
    'obesity': ['obesity', 'obese', 'percent of adults with obesity']
}

def label_indicator(question):
    if not isinstance(question, str): return None
    q = question.lower()
    for ind, kws in indicator_keywords.items():
        for kw in kws:
            if kw in q: return ind
    return None

df['indicator'] = df['Question'].apply(label_indicator)
df['Data_Value'] = pd.to_numeric(df['Data_Value'], errors='coerce')
labeled = df.dropna(subset=['indicator', 'Data_Value']).copy()

# Filter for latest year (2021)
latest_df = labeled[labeled['YearStart'] == 2021].copy()

# 3. BUILD LOOKUP TREE (Demographics)
# ---------------------------------------------------------
# Structure: State -> Category (Age/Income) -> Group -> Obesity Rate
print("Building Demographic Lookup Tree...")
lookup_tree = {}

# Filter only obesity rows for the tree
obesity_rows = latest_df[latest_df['indicator'] == 'obesity']

for state in obesity_rows['LocationDesc'].unique():
    state_df = obesity_rows[obesity_rows['LocationDesc'] == state]
    lookup_tree[state] = {}

    for cat in state_df['StratificationCategory1'].unique():
        # Skip "Total" category to keep it clean, or keep it if you want defaults
        cat_df = state_df[state_df['StratificationCategory1'] == cat]
        lookup_tree[state][cat] = {}

        for _, row in cat_df.iterrows():
            group = row['Stratification1']
            val = row['Data_Value']
            lookup_tree[state][cat][group] = val

# 4. TRAIN MODEL (Calculate Coefficients)
# ---------------------------------------------------------
print("Training Prediction Model...")
# Pivot for training
pivot = latest_df.pivot_table(
    index=['LocationDesc', 'StratificationCategory1', 'Stratification1'],
    columns='indicator',
    values='Data_Value'
).reset_index()

model_data = pivot.dropna(subset=['obesity', 'no_exercise', 'low_fruit'])

X = model_data[['low_fruit', 'no_exercise']]
y = model_data['obesity']

reg = LinearRegression()
reg.fit(X, y)

# 5. EXPORT FINAL JSON
# ---------------------------------------------------------
final_output = {
    "model_weights": {
        "intercept": reg.intercept_,
        "low_fruit": reg.coef_[0],
        "no_exercise": reg.coef_[1]
    },
    "demographics": lookup_tree
}

filename = 'obesity_project_data.json'
with open(filename, 'w') as f:
    json.dump(final_output, f, indent=4)

print(f"✅ DONE! Download '{filename}' and put it in your React 'src' folder.")
print(f"Model Intercept: {reg.intercept_:.2f}")
files.download(filename)

🚀 Starting Data Processing...
Building Demographic Lookup Tree...
Training Prediction Model...
✅ DONE! Download 'obesity_project_data.json' and put it in your React 'src' folder.
Model Intercept: 14.36


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import pickle
import json

print("🚀 Starting Training with Cleaned Data...")

# 1. LOAD CLEANED DATA
# ---------------------------------------------------------
try:
    # Loading the file you generated in the previous step
    df = pd.read_csv('clean_obesity_demographics.csv')
    print("✅ Loaded 'clean_obesity_demographics.csv'")
except FileNotFoundError:
    print("❌ Error: 'clean_obesity_demographics.csv' not found. Please run your cleaning code block first!")
    raise

# 2. PREPARE DATA
# ---------------------------------------------------------
# The file is already pivoted, just drop rows with missing values for the model
model_data = df.dropna(subset=['obesity', 'no_exercise', 'low_fruit'])

X = model_data[['low_fruit', 'no_exercise']]
y = model_data['obesity']

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. TRAIN & EVALUATE
# ---------------------------------------------------------
print("🧠 Training Model...")
reg = LinearRegression()
reg.fit(X_train, y_train)

# Metrics
predictions = reg.predict(X_test)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print("-" * 30)
print(f"✅ Model Accuracy (R²): {r2:.4f}")
print(f"📉 Mean Absolute Error: {mae:.2f}%")
print("-" * 30)

# 4. EXPORT FILES
# ---------------------------------------------------------
# Save .pkl (for Python GUI)
with open('obesity_model.pkl', 'wb') as f:
    pickle.dump(reg, f)

# Save .json (for React App)
# We reconstruct the tree from the cleaned dataframe
lookup_tree = {}

# Iterate over the dataframe to build the State -> Category -> Group structure
for _, row in df.iterrows():
    # Skip rows where obesity data is missing
    if pd.isna(row['obesity']): continue

    state = row['state']
    cat = row['category']
    group = row['group']
    val = row['obesity']

    if state not in lookup_tree: lookup_tree[state] = {}
    if cat not in lookup_tree[state]: lookup_tree[state][cat] = {}

    lookup_tree[state][cat][group] = val

final_output = {
    "model_weights": {
        "intercept": reg.intercept_,
        "low_fruit": reg.coef_[0],
        "no_exercise": reg.coef_[1]
    },
    "demographics": lookup_tree
}

with open('obesity_project_data.json', 'w') as f:
    json.dump(final_output, f, indent=4)

print("💾 Saved: obesity_model.pkl")
print("💾 Saved: obesity_project_data.json (Download this for React)")

🚀 Starting Training with Cleaned Data...
✅ Loaded 'clean_obesity_demographics.csv'
🧠 Training Model...
------------------------------
✅ Model Accuracy (R²): 0.1940
📉 Mean Absolute Error: 4.89%
------------------------------
💾 Saved: obesity_model.pkl
💾 Saved: obesity_project_data.json (Download this for React)


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import numpy as np

print("🖥️ Launching Advanced GUI...")

# 1. LOAD DATA (JSON)
# We need the JSON to look up the "Base Rates" for demographics
try:
    with open('obesity_project_data.json', 'r') as f:
        project_data = json.load(f)
    print("✅ Project Data Loaded.")
except FileNotFoundError:
    print("❌ Error: 'obesity_project_data.json' not found. Run Part 1 first.")
    raise

demographics = project_data['demographics']
weights = project_data['model_weights']

# 2. CREATE WIDGETS
style = {'description_width': 'initial'}

# --- Dropdowns ---
# Sort states for better UX
state_options = sorted(list(demographics.keys()))

state_dropdown = widgets.Dropdown(
    options=state_options,
    value=state_options[0], # Default to first state
    description='📍 State:',
    style=style,
    layout=widgets.Layout(width='90%')
)

category_dropdown = widgets.Dropdown(
    description='📂 Category:',
    style=style,
    layout=widgets.Layout(width='90%')
)

group_dropdown = widgets.Dropdown(
    description='👥 Group:',
    style=style,
    layout=widgets.Layout(width='90%')
)

# --- Sliders ---
fruit_slider = widgets.IntSlider(
    value=50, min=0, max=100,
    description='🍎 Low Fruit Intake (0-100%):', style=style, layout=widgets.Layout(width='90%')
)

exercise_slider = widgets.IntSlider(
    value=50, min=0, max=100,
    description='🏃‍♂️ Physical Inactivity (0-100%):', style=style, layout=widgets.Layout(width='90%')
)

btn_predict = widgets.Button(
    description="Analyze Risk",
    button_style='primary',
    layout=widgets.Layout(width='100%', margin='20px 0')
)

output_box = widgets.Output()

# 3. DYNAMIC LOGIC (Cascading Dropdowns)
def update_categories(*args):
    """When State changes, update Categories"""
    state = state_dropdown.value
    cats = list(demographics[state].keys())
    category_dropdown.options = cats
    # Default to 'Age (years)' if available, else first one
    if 'Age (years)' in cats:
        category_dropdown.value = 'Age (years)'
    else:
        category_dropdown.value = cats[0]

def update_groups(*args):
    """When Category changes, update Groups"""
    state = state_dropdown.value
    cat = category_dropdown.value
    if state and cat:
        grps = list(demographics[state][cat].keys())
        group_dropdown.options = grps
        group_dropdown.value = grps[0]

# Attach observers
state_dropdown.observe(update_categories, 'value')
category_dropdown.observe(update_groups, 'value')

# Initialize defaults
update_categories()
update_groups()

# 4. PREDICTION LOGIC (Matching React App)
def on_click_predict(b):
    with output_box:
        clear_output()

        # 1. Get Base Rate from JSON
        state = state_dropdown.value
        cat = category_dropdown.value
        group = group_dropdown.value

        try:
            base_rate = demographics[state][cat][group]
        except KeyError:
            print("⚠️ Data missing for this group. Using default.")
            base_rate = 30.0

        # 2. Calculate Lifestyle Impact
        # Logic: (Input - 50) * (Weight / Scaling Factor)
        # This matches the React logic we wrote earlier
        fruit_score = fruit_slider.value
        exercise_score = exercise_slider.value

        fruit_impact = (fruit_score - 50) * (weights['low_fruit'] / 50)
        exercise_impact = (exercise_score - 50) * (weights['no_exercise'] / 50)

        predicted_risk = base_rate + fruit_impact + exercise_impact

        # 3. Visualize
        color = "#27ae60" # Green
        if predicted_risk > 35: color = "#c0392b" # Red
        elif predicted_risk > 29: color = "#f39c12" # Orange

        display(widgets.HTML(f"""
        <div style="background: #fff; padding: 20px; border-radius: 10px; border-left: 5px solid {color}; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <h3 style="margin:0; color: #7f8c8d;">Result for: {group} ({state})</h3>
            <hr style="border: 0; border-top: 1px solid #eee; margin: 10px 0;">
            <p>Demographic Base Rate: <b>{base_rate:.1f}%</b></p>
            <p>Lifestyle Adjustment: <b style="color:{color}">{fruit_impact + exercise_impact:+.1f}%</b></p>
            <h1 style="color: {color}; margin: 10px 0;">{predicted_risk:.1f}%</h1>
            <span style="background:{color}; color:white; padding: 3px 8px; border-radius:4px; font-size:12px;">ESTIMATED OBESITY RISK</span>
        </div>
        """))

btn_predict.on_click(on_click_predict)

# 5. LAYOUT
ui = widgets.VBox([
    widgets.HTML("<h2 style='color:#2c3e50'>🥑 Advanced Obesity Risk Simulator</h2>"),
    widgets.HTML("<b>1. Select Demographic Profile:</b>"),
    state_dropdown,
    category_dropdown,
    group_dropdown,
    widgets.HTML("<br><b>2. Set Lifestyle Habits:</b>"),
    fruit_slider,
    exercise_slider,
    btn_predict,
    output_box
], layout=widgets.Layout(padding='20px', border='1px solid #ddd', width='600px'))

display(ui)

🖥️ Launching Advanced GUI...
✅ Project Data Loaded.
